In [1]:
import numpy as np
from tensorflow.keras.utils import to_categorical
from tqdm import tqdm
import numpy as np
import pandas as pd
import os
from PIL import Image
import matplotlib.pyplot as plt
import random
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras import layers, models
from tensorflow.keras import optimizers
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import Sequential, layers
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras import regularizers, Sequential, layers
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping

Init Plugin
Init Graph Optimizer
Init Kernel


In [2]:
df = pd.read_csv('../speech_emotion_reco/data/merged_dataset.csv')

In [3]:
df.emotion.value_counts()

happy       1924
sad         1923
fear        1923
disgust     1923
angry       1923
neutral     1895
surprise     452
unknown      200
Name: emotion, dtype: int64

In [4]:
df['savee'] = df['path'].apply(lambda x: 1 if 'savee' in x else 0)
df['crema'] = df['path'].apply(lambda x: 1 if 'crema' in x else 0)
df['ravdess'] = df['path'].apply(lambda x: 1 if 'ravdess' in x else 0)
df['tess'] = df['path'].apply(lambda x: 1 if 'tess' in x else 0)

In [5]:
df.crema.value_counts()

1    7443
0    4720
Name: crema, dtype: int64

In [6]:
crema_df = df[df['crema'] == 1]

In [7]:
crema_df.emotion.value_counts()

happy      1272
angry      1271
disgust    1271
sad        1271
fear       1271
neutral    1087
Name: emotion, dtype: int64

In [8]:
crema_df

,Unnamed: 0,gender,emotion,path,duration,savee,crema,ravdess,tess
4720,4720,male,angry,../speech_emotion_reco/data/crema/1022_ITS_ANG...,2.435782,0,1,0,0
4721,4721,female,angry,../speech_emotion_reco/data/crema/1037_ITS_ANG...,3.003039,0,1,0,0
4722,4722,female,neutral,../speech_emotion_reco/data/crema/1060_ITS_NEU...,2.402404,0,1,0,0
4723,4723,female,neutral,../speech_emotion_reco/data/crema/1075_ITS_NEU...,2.435782,0,1,0,0
4724,4724,female,disgust,../speech_emotion_reco/data/crema/1073_IOM_DIS...,2.869569,0,1,0,0
...,...,...,...,...,...,...,...,...,...
12158,12158,female,angry,../speech_emotion_reco/data/crema/1089_WSI_ANG...,2.268980,0,1,0,0
12159,12159,female,angry,../speech_emotion_reco/data/crema/1025_IWW_ANG...,2.002041,0,1,0,0
12160,12160,female,angry,../speech_emotion_reco/data/crema/1030_IWW_ANG...,2.369025,0,1,0,0
12161,12161,male,angry,../speech_emotion_reco/data/crema/1019_TIE_ANG...,3.303356,0,1,0,0


In [9]:
df.shape

(12163, 9)

In [10]:
df.drop(columns=['Unnamed: 0'], inplace=True)

In [11]:
unknown_indices = list(df[df['emotion'] == 'unknown'].index)

In [12]:
def load_data(reduction_factor=0.10):
    unknown_indices = list(df[df['crema'] == 0].index)
    unknown_path = [f'../speech_emotion_reco/data/images/{i}.png' for i in unknown_indices]
    data_path = '../speech_emotion_reco/data/images/'
    classes = {'happy': 0, 
               'sad': 1,
               'fear': 2,
               'disgust': 3,
               'angry': 4,
               'neutral': 5,
               'surprise': 6,
               'unknown': 7}
    imgs = []
    labels = []
    images_path = [os.path.join(data_path, path) for path in os.listdir(data_path) if path.find('.png') > 0]
    known_images_path = [path for path in images_path if path not in unknown_path]
    index = 4720 
    for path in known_images_path:
        image = Image.open(path).convert('RGB')
        imgs.append(np.array(image))
        labels.append(classes[df.loc[index, 'emotion']])
        index += 1
    print(set(labels))
        
    X = np.array(imgs)
    num_classes = len(set(labels))
    y = to_categorical(labels, num_classes)

    return X, y

#random.sample(known_images_path, int(len(known_images_path)*reduction_factor))

In [13]:
X, y = load_data(reduction_factor=1)

{0, 1, 2, 3, 4, 5}


In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=69)

In [15]:
X.shape

(7443, 288, 432, 3)

In [16]:
y.shape

(7443, 6)

In [17]:
def load_model():
    model = VGG16(weights="imagenet", include_top=False, input_shape=X_train[0].shape)
    return model

In [18]:
model = load_model()
model.summary()

2021-08-25 11:45:27.233872: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-08-25 11:45:27.242544: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-08-25 11:45:27.243939: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 288, 432, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 288, 432, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 288, 432, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 144, 216, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 144, 216, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 144, 216, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (N

In [19]:
def set_nontrainable_layers(model):
    # Set the first layers to be untrainable
    model.trainable = False
    
    return model

model = set_nontrainable_layers(model)

model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 288, 432, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 288, 432, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 288, 432, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 144, 216, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 144, 216, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 144, 216, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 72, 108, 128)      0     

In [20]:
def add_last_layers(model):
    '''Take a pre-trained model, set its parameters as non-trainables, and add additional trainable layers on top'''
    base_model = set_nontrainable_layers(model)
    flatten_layer = layers.Flatten()
    dense_layer = layers.Dense(500, activation='relu')
    dropout_layer = layers.Dropout(0.2)
    prediction_layer = layers.Dense(6, activation='softmax')
    
    
    model = models.Sequential([
        base_model,
        flatten_layer,
        dense_layer,
        dropout_layer,
        prediction_layer
    ])
    return model

In [21]:
model = add_last_layers(model)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 9, 13, 512)        14714688  
_________________________________________________________________
flatten (Flatten)            (None, 59904)             0         
_________________________________________________________________
dense (Dense)                (None, 500)               29952500  
_________________________________________________________________
dropout (Dropout)            (None, 500)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 3006      
Total params: 44,670,194
Trainable params: 29,955,506
Non-trainable params: 14,714,688
_________________________________________________________________


In [22]:
def build_model():  
    model = load_model()
    model = add_last_layers(model)
    
    opt = optimizers.Adam(learning_rate=1e-4)
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    return model

In [23]:
model = build_model()

In [24]:
X_train = preprocess_input(X_train) 
X_test = preprocess_input(X_test)

In [25]:
X_train.shape

(5954, 288, 432, 3)

In [26]:
y_train.shape

(5954, 6)

In [ ]:
es = EarlyStopping(monitor='val_accuracy', mode='max', patience=10, verbose=1, restore_best_weights=True)

history = model.fit(X_train, y_train, 
                    validation_split=0.2, 
                    epochs=50, 
                    batch_size=16, 
                    callbacks=[es])

2021-08-25 11:46:57.223855: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/50


2021-08-25 11:46:57.998853: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


298/298 [==============================] - ETA: 0s - loss: 2.8798 - accuracy: 0.1709

2021-08-25 11:51:50.145624: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


298/298 [==============================] - 370s 1s/step - loss: 2.8798 - accuracy: 0.1709 - val_loss: 1.7991 - val_accuracy: 0.1797
Epoch 2/50
298/298 [==============================] - 453s 2s/step - loss: 1.7940 - accuracy: 0.1862 - val_loss: 1.8036 - val_accuracy: 0.1646
Epoch 3/50
298/298 [==============================] - 458s 2s/step - loss: 1.7808 - accuracy: 0.2049 - val_loss: 1.8031 - val_accuracy: 0.1746
Epoch 4/50
298/298 [==============================] - 497s 2s/step - loss: 1.7670 - accuracy: 0.2076 - val_loss: 1.7997 - val_accuracy: 0.1755
Epoch 5/50
298/298 [==============================] - 569s 2s/step - loss: 1.7519 - accuracy: 0.2221 - val_loss: 1.8199 - val_accuracy: 0.1713
Epoch 6/50
298/298 [==============================] - 535s 2s/step - loss: 1.7079 - accuracy: 0.2482 - val_loss: 1.8630 - val_accuracy: 0.1788
Epoch 7/50
298/298 [==============================] - 551s 2s/step - loss: 1.6711 - accuracy: 0.2710 - val_loss: 1.8699 - val_accuracy: 0.1713
Epoch 8/50

In [ ]:
def load_own_model():

    model = Sequential()
    model.add(Rescaling(1./255, input_shape=X_train[0].shape))

    model.add(layers.Conv2D(32, kernel_size=6, activation="relu"))
    model.add(layers.MaxPooling2D(3))

    model.add(layers.Flatten())
    model.add(layers.Dense(100, activation='relu'))
    model.add(layers.Dense(6, activation='softmax'))
    
    return model

model = load_own_model()
model.summary()

from tensorflow.keras.callbacks import EarlyStopping

model = load_own_model()
model = compile_model(model)

es = EarlyStopping(patience=10, verbose=2)

history = model.fit(X_train, y_train, 
                            validation_split=0.3,
                            epochs=100, 
                            batch_size=64)